In [1]:
%reload_ext autoreload
%autoreload 2

setup_already_done = False

In [2]:
import sys
import os

In [3]:
IN_COLAB = 'google.colab' in sys.modules
IN_COLAB

True

In [4]:
%cd VideoPrediction_MovingMNIST/

/content/VideoPrediction_MovingMNIST


In [5]:
if not setup_already_done:
    if(IN_COLAB):
        !git clone https://github.com/Silver0x10/VideoPrediction_MovingMNIST.git
        %cd VideoPrediction_MovingMNIST
    else:
        %cd ..
    if(not os.path.exists("data/mnist_test_seq.npy")):
        if(not os.path.exists("data")):
            %mkdir data
        %cd data
        !wget -q https://www.cs.toronto.edu/~nitish/unsupervised_video/mnist_test_seq.npy
        %cd ..

    %pip install -qr requirements.txt  --quiet
    setup_already_done = True

Cloning into 'VideoPrediction_MovingMNIST'...
remote: Enumerating objects: 135, done.
remote: Counting objects: 100% (135/135), done.
remote: Compressing objects: 100% (87/87), done.
remote: Total 135 (delta 72), reused 95 (delta 40), pack-reused 0
Receiving objects: 100% (135/135), 75.60 KiB | 4.72 MiB/s, done.
Resolving deltas: 100% (72/72), done.
/content/VideoPrediction_MovingMNIST/VideoPrediction_MovingMNIST
/content/VideoPrediction_MovingMNIST/VideoPrediction_MovingMNIST/data
/content/VideoPrediction_MovingMNIST/VideoPrediction_MovingMNIST


In [6]:
# import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader,random_split
# from torch import optim, nn, utils, Tensor
# import torchvision
# from torchvision.transforms import ToTensor
import lightning.pytorch as pl

In [7]:
# !nvidia-smi

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [8]:
from src.MovingMNIST import MovingMNIST

dataset = MovingMNIST(data_path="data/mnist_test_seq.npy")
train_set, validation_set, test_set = random_split(dataset,[8000, 1000, 1000],
                                                   generator=torch.Generator().manual_seed(42))
# Visualization Test
dataset.visualize(0)

In [9]:
loader = DataLoader(dataset)
training_dataloader = DataLoader(train_set)
validation_dataloader = DataLoader(validation_set)
test_dataloader = DataLoader(test_set)

In [14]:
from src.simpleLSTM import simpleLSTM
import src.Modules as mod
from src.parameters import C


model = simpleLSTM()


encoder = mod.Encoder(C,3)
decoder = mod.Decoder(C,3)
model_2 = mod.PlEncoderDecoder(encoder,decoder)

In [18]:
# for data in loader:
#     print(model.training_step(data,0))
#     break

In [13]:
%pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.4/209.4 kB 21.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.1 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=06c3c1e4d5b87aaf713c14bc24f2f0abf1451d8960386e16600458564f95c0c5
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools


In [11]:
from lightning.pytorch.loggers.wandb import WandbLogger
logger = WandbLogger()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [12]:
# trainer = pl.Trainer(limit_train_batches=1, max_epochs=1, accelerator="cpu")
# trainer = pl.Trainer(max_epochs=1, limit_train_batches=100, accelerator="cpu")
# trainer = pl.Trainer(max_epochs=5, limit_train_batches=100)
trainer = pl.Trainer(max_epochs=10, accelerator="gpu", logger=logger)

INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [15]:
trainer.fit(model=model_2, train_dataloaders=training_dataloader, val_dataloaders=validation_dataloader)

/usr/local/lib/python3.10/dist-packages/lightning/pytorch/trainer/configuration_validator.py:68: UserWarning: You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.
  rank_zero_warn("You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.")
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name    | Type    | Params
------------------------------------
0 | lstm    | LSTM    | 526 K 
1 | encoder | Encoder | 40    
2 | decoder | Decoder | 40    
------------------------------------
526 K     Trainable params
0         Non-trainable params
526 K     Total params
2.106     Total estimated model params size (MB)
INFO:lightning.pytorch.callbacks.model_summary:
  | Name    | Type    | Params
------------------------------------
0 | lstm    | LSTM    | 526 K 
1 | encoder | Encoder | 40    
2 | decoder | Decoder | 40    
-----------------------------

Training: 0it [00:00, ?it/s]

/content/VideoPrediction_MovingMNIST/VideoPrediction_MovingMNIST/src/Modules.py:134: UserWarning: Using a target size (torch.Size([1, 64, 64])) that is different to the input size (torch.Size([1, 1, 64, 64])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = nn.functional.mse_loss(out, y)
INFO: `Trainer.fit` stopped: `max_epochs=10` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


In [18]:
trainer.test(model, dataloaders=test_dataloader)

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │     2734.07861328125      │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 2734.07861328125}]

In [ ]:
# !zip -r ../logs lightning_logs/

In [ ]:
# %reload_ext tensorboard
# %tensorboard --logdir=lightning_logs/ #--port 6007

In [ ]:
# import gc
# print(torch.cuda.list_gpu_processes())
# gc.collect()
# torch.cuda.empty_cache()